In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import nltk,string
from gensim import corpora
import csv
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np
from sklearn.model_selection import train_test_split

C:\Users\Siyuan\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Siyuan\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open("amazon_review_large.csv", "r") as f:
     
    csvfile = csv.reader(f, delimiter=',')

    label,text=zip(*csvfile)

# convert tuple to list
text=list(text)
label=list(label)
for i, x in enumerate(label):
    if x == '1': label[i] = 0
    else: label[i]=1

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

# set the maximum number of words to be used
MAX_NB_WORDS=10000

# set sentence/document length
MAX_DOC_LEN=500

# get a Keras tokenizer
# https://keras.io/preprocessing/text/
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)

# get the mapping between word and its index
voc=tokenizer.word_index

# convert each document to a list of word index as a sequence
sequences = tokenizer.texts_to_sequences(text)

# pad all sequences into the same length 
# if a sentence is longer than maxlen, pad it in the right
# if a sentence is shorter than maxlen, truncate it in the right
padded_sequences = pad_sequences(sequences, \
                                 maxlen=MAX_DOC_LEN, \
                                 padding='post', \
                                 truncating='post')



In [4]:
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, \
Dropout, Activation, Input, Flatten, Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
              
def cnn_model(FILTER_SIZES, \
              # filter sizes as a list
              MAX_NB_WORDS, \
              # total number of words
              MAX_DOC_LEN, \
              # max words in a doc
              NUM_OUTPUT_UNITS=1, \
              # number of output units
              EMBEDDING_DIM=200, \
              # word vector dimension
              NUM_FILTERS=64, \
              # number of filters for all size
              DROP_OUT=0.5, \
              # dropout rate
              PRETRAINED_WORD_VECTOR=None,\
              # Whether to use pretrained word vectors
              LAM=0.01):            
              # regularization coefficient
    
    main_input = Input(shape=(MAX_DOC_LEN,), \
                       dtype='int32', name='main_input')
    
    if PRETRAINED_WORD_VECTOR is not None:
        embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                        output_dim=EMBEDDING_DIM, \
                        input_length=MAX_DOC_LEN, \
                        weights=[PRETRAINED_WORD_VECTOR],\
                        trainable=False,\
                        name='embedding')(main_input)
    else:
        embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                        output_dim=EMBEDDING_DIM, \
                        input_length=MAX_DOC_LEN, \
                        name='embedding')(main_input)
    # add convolution-pooling-flat block
    conv_blocks = []
    for f in FILTER_SIZES:
        conv = Conv1D(filters=NUM_FILTERS, kernel_size=f, \
                      activation='relu', name='conv_'+str(f))(embed_1)
        conv = MaxPooling1D(MAX_DOC_LEN-f+1, name='max_'+str(f))(conv)
        conv = Flatten(name='flat_'+str(f))(conv)
        conv_blocks.append(conv)

    z=Concatenate(name='concate')(conv_blocks)
    drop=Dropout(rate=DROP_OUT, name='dropout')(z)

    dense = Dense(192, activation='relu',\
                    kernel_regularizer=l2(LAM),name='dense')(drop)
    preds = Dense(NUM_OUTPUT_UNITS, activation='sigmoid', name='output')(dense)
    model = Model(inputs=main_input, outputs=preds)
    
    model.compile(loss="binary_crossentropy", \
              optimizer="adam", metrics=["accuracy"]) 
    
    return model

In [6]:
NUM_OUTPUT_UNITS=2
MAX_NB_WORDS=10000
MAX_DOC_LEN=500
# filters on bigrams, trigrams, and 4-grams
FILTER_SIZES=[2,3,4]
BEST_MODEL_FILEPATH='best_model'

BATCH_SIZES = 64
NUM_EPOCHES = 20

X_train, X_test, Y_train, Y_test = train_test_split(\
                padded_sequences, label, test_size=0.3, random_state=0)

model=cnn_model(FILTER_SIZES, MAX_NB_WORDS, MAX_DOC_LEN)

earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=2, mode='min')
checkpoint = ModelCheckpoint(BEST_MODEL_FILEPATH, monitor='val_acc', \
                             verbose=2, save_best_only=True, mode='max')
    
training=model.fit(X_train, Y_train, \
          batch_size=BATCH_SIZES, epochs=NUM_EPOCHES, \
          callbacks=[earlyStopping, checkpoint],\
          validation_data=[X_test, Y_test], verbose=2)

Train on 14000 samples, validate on 6000 samples
Epoch 1/20
 - 88s - loss: 0.9119 - acc: 0.6996 - val_loss: 0.4123 - val_acc: 0.8365

Epoch 00001: val_acc improved from -inf to 0.83650, saving model to best_model
Epoch 2/20
 - 85s - loss: 0.3560 - acc: 0.8598 - val_loss: 0.3999 - val_acc: 0.8363

Epoch 00002: val_acc did not improve from 0.83650
Epoch 3/20
 - 83s - loss: 0.2257 - acc: 0.9284 - val_loss: 0.3749 - val_acc: 0.8593

Epoch 00003: val_acc improved from 0.83650 to 0.85933, saving model to best_model
Epoch 4/20
 - 88s - loss: 0.1431 - acc: 0.9623 - val_loss: 0.4140 - val_acc: 0.8568

Epoch 00004: val_acc did not improve from 0.85933
Epoch 00004: early stopping


In [7]:
model.load_weights("best_model")

# evaluate the model
scores = model.evaluate(padded_sequences, label, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 94.42%


[[ 0.94837356]
 [ 0.02075764]
 [ 0.40209559]
 [ 0.63034117]
 [ 0.18376663]
 [ 0.89941788]
 [ 0.48391566]
 [ 0.85834521]
 [ 0.84794623]
 [ 0.51868761]]
